<a href="https://colab.research.google.com/github/diegomrodrigues/generative_models_experiments/blob/main/Full%20VAE%20Training%20on%20CIFAR-10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers huggingface_hub tensorboardX --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.6 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import numpy as np
from huggingface_hub import HfApi, notebook_login
from huggingface_hub import HFSummaryWriter
import os
from transformers import Trainer, TrainingArguments
import io

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
batch_size = 1024
latent_dim = 256
learning_rate = 1e-3
num_epochs = 50

repo_name = "diegomrodrigues/vae-cifar10-experiment"  # Replace with your Hugging Face username

# Data loading and preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Wrapper for CIFAR-10 dataset to make it compatible with Hugging Face Trainer
class CIFAR10Wrapper(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, index):
        image, _ = self.dataset[index]
        return {"pixel_values": image}

    def __len__(self):
        return len(self.dataset)

train_dataset = CIFAR10Wrapper(datasets.CIFAR10(root='./data', train=True, download=True, transform=transform))

# VAE model
class VAE(nn.Module):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(256, latent_dim)
        self.fc_logvar = nn.Linear(256, latent_dim)

        # Decoder
        self.decoder_input = nn.Linear(latent_dim, 128 * 4 * 4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = self.decoder_input(z)
        h = h.view(-1, 128, 4, 4)
        return self.decoder(h)

    def forward(self, pixel_values):
        mu, logvar = self.encode(pixel_values)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Initialize model
model = VAE(latent_dim).to(device)

# Loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD, BCE, KLD

# Set up Hugging Face SummaryWriter
hf_writer = HFSummaryWriter(repo_id=repo_name)

# Custom Trainer class for VAE
class VAETrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.hf_writer = hf_writer

    def compute_loss(self, model, inputs, return_outputs=False):
        pixel_values = inputs['pixel_values']
        recon_batch, mu, logvar = model(pixel_values)
        loss, bce, kld = loss_function(recon_batch, pixel_values, mu, logvar)

        # Log metrics
        self.log_metrics(loss, bce, kld)

        return (loss, (loss, recon_batch)) if return_outputs else loss

    def log_metrics(self, loss, bce, kld):
        if self.state.global_step % 100 == 0:
            self.hf_writer.add_scalar('Loss/total', loss.item(), self.state.global_step)
            self.hf_writer.add_scalar('Loss/BCE', bce.item(), self.state.global_step)
            self.hf_writer.add_scalar('Loss/KLD', kld.item(), self.state.global_step)
            self.hf_writer.add_scalar('Loss/ELBO', -loss.item(), self.state.global_step)

    def log(self, logs):
        super().log(logs)
        if self.state.global_step % 100 == 0:
            self.save_samples(self.state.global_step)

    def save_samples(self, step):
        model.eval()
        with torch.no_grad():
            sample = torch.randn(64, latent_dim).to(device)
            sample = model.decode(sample).cpu()
            img_grid = make_grid(sample, normalize=True, nrow=8)

            # Save image to BytesIO object
            buf = io.BytesIO()
            plt.figure(figsize=(10, 10))
            plt.imshow(np.transpose(img_grid, (1, 2, 0)))
            plt.axis('off')
            plt.title(f'Generated samples - Step {step}')
            plt.savefig(buf, format='png')
            plt.close()
            buf.seek(0)

            # Log image to Hugging Face
            self.hf_writer.add_image('Generated samples', img_grid, step, dataformats='CHW')
            self.hf_writer.add_figure('Generated samples (matplotlib)', plt.gcf(), step)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=1000,
    learning_rate=learning_rate,
    remove_unused_columns=False,
)

# Initialize trainer
trainer = VAETrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Create samples directory
os.makedirs('samples', exist_ok=True)

# Train the model
trainer.train()

print("Training complete!")

# Close Hugging Face SummaryWriter
hf_writer.close()

# Push the model to the Hugging Face Hub
trainer.push_to_hub(commit_message="Upload VAE model")

Using device: cuda


100%|██████████| 170498071/170498071 [00:04<00:00, 42215256.21it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_experimental.py:58: UserWarning: 'HFSummaryWriter' is experimental and might be subject to breaking changes in the future. You can disable this warning by setting `HF_HUB_DISABLE_EXPERIMENTAL_WARNING=1` as environment variable.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Step,Training Loss
100,518288.880000
200,366205.120000
300,311372.960000
400,278783.440000
500,263293.580000
600,252716.200000
700,243776.640000
800,237938.540000
900,234418.480000
1000,232251.920000


Training complete!


AttributeError: 'VAE' object has no attribute 'config'

In [8]:
!pip install pytorch-msssim torchmetrics[image] torch-fidelity --quiet

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import numpy as np
from huggingface_hub import HfApi, notebook_login
from huggingface_hub import HFSummaryWriter
import os
from transformers import Trainer, TrainingArguments
import io
from pytorch_msssim import ssim
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
batch_size = 128
latent_dim = 256
learning_rate = 1e-3
num_epochs = 50
warmup_steps = 1000
max_grad_norm = 1.0

repo_name = "diegomrodrigues/vae-cifar10-experiment"

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

class CIFAR10Wrapper(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, index):
        image, _ = self.dataset[index]
        return {"pixel_values": image}

    def __len__(self):
        return len(self.dataset)

train_dataset = CIFAR10Wrapper(datasets.CIFAR10(root='./data', train=True, download=True, transform=transform))

class VAE(nn.Module):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256 * 2 * 2, 512),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(512, latent_dim)
        self.fc_logvar = nn.Linear(512, latent_dim)

        # Decoder
        self.decoder_input = nn.Linear(latent_dim, 256 * 2 * 2)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = self.decoder_input(z)
        h = h.view(-1, 256, 2, 2)
        return self.decoder(h)

    def forward(self, pixel_values):
        mu, logvar = self.encode(pixel_values)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

model = VAE(latent_dim).to(device)

def loss_function(recon_x, x, mu, logvar, kl_weight):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD, BCE, KLD

hf_writer = HFSummaryWriter(repo_id=repo_name)

class VAETrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.hf_writer = hf_writer
        self.kl_weight = 0.0

    def compute_loss(self, model, inputs, return_outputs=False):
        pixel_values = inputs['pixel_values']
        recon_batch, mu, logvar = model(pixel_values)
        loss, bce, kld = loss_function(recon_batch, pixel_values, mu, logvar, self.kl_weight)

        # KL annealing
        self.kl_weight = min(self.kl_weight + 1 / (10 * len(self.train_dataset)), 1.0)

        # Log metrics
        self.log_metrics(loss, bce, kld, recon_batch, pixel_values)

        return (loss, (loss, recon_batch)) if return_outputs else loss

    def log_metrics(self, loss, bce, kld, recon_batch, pixel_values):
        if self.state.global_step % 100 == 0:
            self.hf_writer.add_scalar('Loss/total', loss.item(), self.state.global_step)
            self.hf_writer.add_scalar('Loss/BCE', bce.item(), self.state.global_step)
            self.hf_writer.add_scalar('Loss/KLD', kld.item(), self.state.global_step)
            self.hf_writer.add_scalar('Loss/ELBO', -loss.item(), self.state.global_step)
            self.hf_writer.add_scalar('KL_weight', self.kl_weight, self.state.global_step)

            # Calculate SSIM
            ssim_val = ssim(recon_batch, pixel_values, data_range=2.0, size_average=True)
            self.hf_writer.add_scalar('Metrics/SSIM', ssim_val.item(), self.state.global_step)

    def log(self, logs):
        super().log(logs)
        if self.state.global_step % 1000 == 0:
            self.save_samples(self.state.global_step)

    def save_samples(self, step):
        model.eval()
        with torch.no_grad():
            sample = torch.randn(64, latent_dim).to(device)
            sample = model.decode(sample).cpu()
            img_grid = make_grid(sample, normalize=True, nrow=8)

            buf = io.BytesIO()
            plt.figure(figsize=(10, 10))
            plt.imshow(np.transpose(img_grid, (1, 2, 0)))
            plt.axis('off')
            plt.title(f'Generated samples - Step {step}')
            plt.savefig(buf, format='png')
            plt.close()
            buf.seek(0)

            self.hf_writer.add_image('Generated samples', img_grid, step, dataformats='CHW')
            self.hf_writer.add_figure('Generated samples (matplotlib)', plt.gcf(), step)

def get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=1000,
    learning_rate=learning_rate,
    remove_unused_columns=False,
    fp16=True,
    gradient_accumulation_steps=2,
    warmup_steps=warmup_steps,
    max_grad_norm=max_grad_norm,
)

optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataset) * num_epochs // (batch_size * training_args.gradient_accumulation_steps)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

trainer = VAETrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    optimizers=(optimizer, scheduler),
)

os.makedirs('samples', exist_ok=True)

trainer.train()

print("Training complete!")

hf_writer.close()

trainer.push_to_hub(commit_message="Upload improved VAE model")

Using device: cuda
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_experimental.py:58: UserWarning: 'HFSummaryWriter' is experimental and might be subject to breaking changes in the future. You can disable this warning by setting `HF_HUB_DISABLE_EXPERIMENTAL_WARNING=1` as environment variable.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
100,145900.040000
200,66875.190000
300,40583.895000
400,33212.032500
500,27485.332500
600,24392.087500
700,22648.275000
800,21270.095000
900,20294.495000
1000,19484.751300


KeyboardInterrupt: 

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import numpy as np
from huggingface_hub import HfApi, notebook_login
from huggingface_hub import HFSummaryWriter
import os
import io
from pytorch_msssim import ssim
import math
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
batch_size = 256
latent_dim = 256
learning_rate = 1e-3
num_epochs = 50
warmup_steps = 1000
max_grad_norm = 1.0

repo_name = "diegomrodrigues/vae-cifar10-experiment"

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

class CIFAR10Wrapper(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, index):
        image, _ = self.dataset[index]
        return {"pixel_values": image}

    def __len__(self):
        return len(self.dataset)

train_dataset = CIFAR10Wrapper(datasets.CIFAR10(root='./data', train=True, download=True, transform=transform))

# Use DataLoader with num_workers and pin_memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

class VAE(nn.Module):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256 * 2 * 2, 512),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(512, latent_dim)
        self.fc_logvar = nn.Linear(512, latent_dim)

        # Decoder
        self.decoder_input = nn.Linear(latent_dim, 256 * 2 * 2)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = self.decoder_input(z)
        h = h.view(-1, 256, 2, 2)
        return self.decoder(h)

    def forward(self, pixel_values):
        mu, logvar = self.encode(pixel_values)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

model = VAE(latent_dim).to(device)

def loss_function(recon_x, x, mu, logvar, kl_weight):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD, BCE, KLD

hf_writer = HFSummaryWriter(repo_id=repo_name)

def get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * num_epochs
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
scaler = GradScaler()

def train_epoch(model, train_loader, optimizer, scheduler, scaler, epoch):
    model.train()
    train_loss = 0
    kl_weight = 0.0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch_idx, batch in enumerate(progress_bar):
        pixel_values = batch['pixel_values'].to(device)
        optimizer.zero_grad()

        with autocast():
            recon_batch, mu, logvar = model(pixel_values)
            loss, bce, kld = loss_function(recon_batch, pixel_values, mu, logvar, kl_weight)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        train_loss += loss.item()
        kl_weight = min(kl_weight + 1 / (10 * len(train_loader)), 1.0)

        if batch_idx % 100 == 0:
            hf_writer.add_scalar('Loss/total', loss.item(), epoch * len(train_loader) + batch_idx)
            hf_writer.add_scalar('Loss/BCE', bce.item(), epoch * len(train_loader) + batch_idx)
            hf_writer.add_scalar('Loss/KLD', kld.item(), epoch * len(train_loader) + batch_idx)
            hf_writer.add_scalar('Loss/ELBO', -loss.item(), epoch * len(train_loader) + batch_idx)
            hf_writer.add_scalar('KL_weight', kl_weight, epoch * len(train_loader) + batch_idx)

            with torch.no_grad():
                ssim_val = ssim(recon_batch.float(), pixel_values.float(), data_range=2.0, size_average=True)
            hf_writer.add_scalar('Metrics/SSIM', ssim_val.item(), epoch * len(train_loader) + batch_idx)

        progress_bar.set_postfix({'loss': loss.item()})

    return train_loss / len(train_loader)

def save_samples(model, epoch):
    model.eval()
    with torch.no_grad():
        sample = torch.randn(64, latent_dim).to(device)
        sample = model.decode(sample).cpu()
        img_grid = make_grid(sample, normalize=True, nrow=8)

        buf = io.BytesIO()
        plt.figure(figsize=(10, 10))
        plt.imshow(np.transpose(img_grid, (1, 2, 0)))
        plt.axis('off')
        plt.title(f'Generated samples - Epoch {epoch+1}')
        plt.savefig(buf, format='png')
        plt.close()
        buf.seek(0)

        hf_writer.add_image('Generated samples', img_grid, epoch, dataformats='CHW')
        hf_writer.add_figure('Generated samples (matplotlib)', plt.gcf(), epoch)

os.makedirs('samples', exist_ok=True)

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, scheduler, scaler, epoch)
    print(f'Epoch: {epoch+1}, Average loss: {train_loss:.4f}')

    if (epoch + 1) % 5 == 0:
        save_samples(model, epoch)

print("Training complete!")

hf_writer.close()

# Save the model
torch.save(model.state_dict(), 'vae_model.pth')

# Push the model to the Hugging Face Hub
api = HfApi()
api.upload_file(
    path_or_fileobj="vae_model.pth",
    path_in_repo="vae_model.pth",
    repo_id=repo_name,
    commit_message=f"Upload VAE model after {num_epochs} epochs"
)

Using device: cuda
Files already downloaded and verified


<ipython-input-15-ff10d15ff5e2>:136: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/50:   0%|          | 0/196 [00:00<?, ?it/s]<ipython-input-15-ff10d15ff5e2>:148: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/50: 100%|██████████| 196/196 [00:16<00:00, 11.66it/s, loss=2.97e+4]


Epoch: 1, Average loss: 237052.2893


Epoch 2/50: 100%|██████████| 196/196 [00:15<00:00, 12.83it/s, loss=1.95e+4]


Epoch: 2, Average loss: 74693.8434


...

Epoch: 41, Average loss: 24974.2792


Epoch 42/50: 100%|██████████| 196/196 [00:15<00:00, 12.57it/s, loss=1.08e+4]


Epoch: 42, Average loss: 24896.7857


Epoch 43/50:  31%|███       | 61/196 [00:04<00:09, 13.74it/s, loss=2.42e+4]


KeyboardInterrupt: 

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
import numpy as np
from huggingface_hub import HfApi, notebook_login
from huggingface_hub import HFSummaryWriter
import os
import io
from pytorch_msssim import ssim
import math
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
batch_size = 256
latent_dim = 256
learning_rate = 1e-3
num_epochs = 50
warmup_steps = 1000
max_grad_norm = 1.0

repo_name = "diegomrodrigues/vae-cifar10-experiment"

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

class CIFAR10Wrapper(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __getitem__(self, index):
        image, _ = self.dataset[index]
        return {"pixel_values": image}

    def __len__(self):
        return len(self.dataset)

train_dataset = CIFAR10Wrapper(datasets.CIFAR10(root='./data', train=True, download=True, transform=transform))
test_dataset = CIFAR10Wrapper(datasets.CIFAR10(root='./data', train=False, download=True, transform=transform))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

class VAE(nn.Module):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
            nn.Linear(256 * 2 * 2, 512),
            nn.LeakyReLU(0.2)
        )
        self.fc_mu = nn.Linear(512, latent_dim)
        self.fc_logvar = nn.Linear(512, latent_dim)

        # Decoder
        self.decoder_input = nn.Linear(latent_dim, 256 * 2 * 2)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = self.decoder_input(z)
        h = h.view(-1, 256, 2, 2)
        return self.decoder(h)

    def forward(self, pixel_values):
        mu, logvar = self.encode(pixel_values)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

model = VAE(latent_dim).to(device)

def loss_function(recon_x, x, mu, logvar, kl_weight):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + kl_weight * KLD, BCE, KLD

hf_writer = HFSummaryWriter(repo_id=repo_name)

def get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * num_epochs
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
scaler = GradScaler()

def train_epoch(model, train_loader, optimizer, scheduler, scaler, epoch):
    model.train()
    train_loss = 0
    kl_weight = 0.0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch_idx, batch in enumerate(progress_bar):
        pixel_values = batch['pixel_values'].to(device)
        optimizer.zero_grad()

        with autocast():
            recon_batch, mu, logvar = model(pixel_values)
            loss, bce, kld = loss_function(recon_batch, pixel_values, mu, logvar, kl_weight)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        train_loss += loss.item()
        kl_weight = min(kl_weight + 1 / (10 * len(train_loader)), 1.0)

        if batch_idx % 100 == 0:
            hf_writer.add_scalar('Loss/total', loss.item(), epoch * len(train_loader) + batch_idx)
            hf_writer.add_scalar('Loss/BCE', bce.item(), epoch * len(train_loader) + batch_idx)
            hf_writer.add_scalar('Loss/KLD', kld.item(), epoch * len(train_loader) + batch_idx)
            hf_writer.add_scalar('Loss/ELBO', -loss.item(), epoch * len(train_loader) + batch_idx)
            hf_writer.add_scalar('KL_weight', kl_weight, epoch * len(train_loader) + batch_idx)

            with torch.no_grad():
                ssim_val = ssim(recon_batch.float(), pixel_values.float(), data_range=2.0, size_average=True)
            hf_writer.add_scalar('Metrics/SSIM', ssim_val.item(), epoch * len(train_loader) + batch_idx)

        progress_bar.set_postfix({'loss': loss.item()})

    return train_loss / len(train_loader)

def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    ssim_sum = 0

    with torch.no_grad():
        for batch in test_loader:
            pixel_values = batch['pixel_values'].to(device)
            recon_batch, mu, logvar = model(pixel_values)
            loss, _, _ = loss_function(recon_batch, pixel_values, mu, logvar, 1.0)
            test_loss += loss.item()

            ssim_val = ssim(recon_batch.float(), pixel_values.float(), data_range=2.0, size_average=True)
            ssim_sum += ssim_val.item()

    test_loss /= len(test_loader)
    ssim_avg = ssim_sum / len(test_loader)

    return test_loss, ssim_avg

def save_samples(model, epoch):
    model.eval()
    with torch.no_grad():
        sample = torch.randn(64, latent_dim).to(device)
        sample = model.decode(sample).cpu()
        img_grid = make_grid(sample, normalize=True, nrow=8)

        buf = io.BytesIO()
        plt.figure(figsize=(10, 10))
        plt.imshow(np.transpose(img_grid, (1, 2, 0)))
        plt.axis('off')
        plt.title(f'Generated samples - Epoch {epoch+1}')
        plt.savefig(buf, format='png')
        plt.close()
        buf.seek(0)

        hf_writer.add_image('Generated samples', img_grid, epoch, dataformats='CHW')
        hf_writer.add_figure('Generated samples (matplotlib)', plt.gcf(), epoch)

os.makedirs('samples', exist_ok=True)

best_ssim = 0
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, scheduler, scaler, epoch)
    test_loss, ssim_avg = evaluate(model, test_loader)

    print(f'Epoch: {epoch+1}, Train loss: {train_loss:.4f}, Test loss: {test_loss:.4f}, SSIM: {ssim_avg:.4f}')

    hf_writer.add_scalar('Epoch/Train_Loss', train_loss, epoch)
    hf_writer.add_scalar('Epoch/Test_Loss', test_loss, epoch)
    hf_writer.add_scalar('Epoch/SSIM', ssim_avg, epoch)

    if ssim_avg > best_ssim:
        best_ssim = ssim_avg
        torch.save(model.state_dict(), 'best_vae_model.pth')

    if (epoch + 1) % 5 == 0:
        save_samples(model, epoch)

print("Training complete!")

hf_writer.close()

# Save the final model
torch.save(model.state_dict(), 'final_vae_model.pth')

# Push the models to the Hugging Face Hub
api = HfApi()
api.upload_file(
    path_or_fileobj="best_vae_model.pth",
    path_in_repo="best_vae_model.pth",
    repo_id=repo_name,
    commit_message=f"Upload best VAE model (SSIM: {best_ssim:.4f})"
)
api.upload_file(
    path_or_fileobj="final_vae_model.pth",
    path_in_repo="final_vae_model.pth",
    repo_id=repo_name,
    commit_message=f"Upload final VAE model after {num_epochs} epochs"
)

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified


<ipython-input-17-b5c248a73fc9>:137: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
Epoch 1/50:   0%|          | 0/196 [00:00<?, ?it/s]<ipython-input-17-b5c248a73fc9>:149: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 1/50: 100%|██████████| 196/196 [00:20<00:00,  9.54it/s, loss=2.53e+4]


Epoch: 1, Train loss: 208892.1422, Test loss: 110205.3153, SSIM: 0.1691


Epoch 2/50: 100%|██████████| 196/196 [00:17<00:00, 11.47it/s, loss=1.91e+4]


Epoch: 2, Train loss: 65264.6842, Test loss: 97657.4616, SSIM: 0.3244


...

Epoch 50/50: 100%|██████████| 196/196 [00:16<00:00, 12.06it/s, loss=8.26e+3]


Epoch: 50, Train loss: 21627.8107, Test loss: 76373.3919, SSIM: 0.6832
Training complete!


best_vae_model.pth:   0%|          | 0.00/9.75M [00:00<?, ?B/s]

final_vae_model.pth:   0%|          | 0.00/9.75M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/diegomrodrigues/vae-cifar10-experiment/commit/437caa1c467be29d584ceaaee0c9c8326bd8c394', commit_message='Upload final VAE model after 50 epochs', commit_description='', oid='437caa1c467be29d584ceaaee0c9c8326bd8c394', pr_url=None, pr_revision=None, pr_num=None)